In [1]:
import jax
import jax.random as random
import jax.numpy as jnp

import flax

from differentials import expression, domain, boundary, initial

In [2]:
dx = lambda u: jax.grad(u, argnums=0)
dt = lambda u: jax.grad(u, argnums=1)

heat = expression(
    lambda u: lambda x, t: dt(u)(x, t) + dx(dx(u))(x, t),
    var=("x", "t"),
    boundaries=(
        # insulated ends u_x(0, t) = 0
        boundary(
            LHS=lambda u: lambda x, t: dx(u)(x, t),
            RHS=lambda u: lambda x, t: 0.0,
            con=(0.0, "t")
        ),
        # insulated end u_x(L, t) = 0
        boundary(
            LHS=lambda u: lambda x, t: dx(u)(x, t),
            RHS=lambda u: lambda x, t: 0.0,
            con=(1.0, "t")
        ),
        # inital function. u(x, 0) = sin(x)
        initial(
            LHS=lambda u: lambda x, t: u(x, t),
            RHS=lambda u: lambda x, t: jnp.sin(x),
            con=("x", 0.0)
        )
    ),
    x=domain(0, 1),
    t=domain(0, 1)
)

In [3]:
def make_pde_loss(expression):
    u_hat, _ = expression.u((4, 4, 4))
    def loss(params):
        # make values
        n = 30  # hyper parameter, mean samples taken
        xs_matrix = expression.matrix(n)
        def rnd_instance_val(x):
            print(x)
            print(*[num for num in x])
            error = expression.loss(
                    lambda x, t: u_hat.apply(params, jnp.array((x, t))),
                    *[num for num in x]
            )
            print("here", error)
            return error

        return jnp.mean(jax.vmap(rnd_instance_val, in_axes=0)(xs_matrix))
    return jax.jit(loss)

In [29]:
import time
from tqdm import tqdm

In [26]:
u_hat, params = heat.u()

x =jnp.array([1.0,2.0])
xs = heat.matrix(10)

#start val
def val(x):
    error = heat.loss(
            lambda x, t: u_hat.apply(params, jnp.array((x, t)))[0],
            x[0], x[1]  # this is for x and t. No better way exists to do this
    )
    return error

def make_loss(expression):
    u_hat, _ = expression.u()
    # hyper param, num of samples per loss
    xs = expression.matrix(10)
    def loss(params):
        def loss_unit(x):
            error = expression.loss(
                lambda x, t: u_hat.apply(params, jnp.array((x, t)))[0],
                x[0], x[1]  # this is for x and t. No better way exists to do this
            )
            return error
        return jnp.max(jax.vmap(loss_unit)(xs))
        return jnp.mean(jax.vmap(loss_unit)(xs))
        # here there is a contention. What loss is better, the worst point tested, or the average point tested
    return jax.jit(loss)

start = time.time()
heat_loss = make_loss(heat)
heat_loss, grads = jax.value_and_grad(heat_loss)(params)
end = time.time()

print(end - start)

print(heat_loss)
print(grads)




5.055876016616821
1.6818761
{'params': {'Dense_0': {'bias': Array([-6.8500056,  5.8860106], dtype=float32), 'kernel': Array([[-3.5157905 ,  2.8558013 ],
       [ 1.4642328 , -0.82544225]], dtype=float32)}, 'Dense_1': {'bias': Array([-2.7510614, -2.0679293, -4.046641 , -2.609736 ], dtype=float32), 'kernel': Array([[-0.38641772, -0.12640245, -0.28966168, -0.28075045],
       [-1.4686193 , -1.3215141 , -1.7179074 , -1.548946  ]],      dtype=float32)}, 'Dense_2': {'bias': Array([ 3.577105  , -2.9246707 , -0.22836965, -0.91325915, -0.2636633 ],      dtype=float32), 'kernel': Array([[-0.5927569 ,  0.65043473,  0.12628253,  0.17360386,  0.14536926],
       [ 0.01639794,  0.08647421,  0.04295447, -0.03324792, -0.10653525],
       [-0.78152436,  0.9497759 ,  0.21021459,  0.20418186,  0.09124723],
       [-0.74358386,  0.8820211 ,  0.18833487,  0.2019265 ,  0.11008704]],      dtype=float32)}, 'Dense_3': {'bias': Array([-2.0352938,  0.6796228, -1.2621865,  1.8939246, -1.1580145],      dtype=float

In [35]:
# timing the loss / average loss with a mean function vs a min function. 

def make_mean_loss(expression):
    u_hat, _ = expression.u()
    # hyper param, num of samples per loss
    xs = expression.matrix(10)
    def loss(params):
        def loss_unit(x):
            error = expression.loss(
                lambda x, t: u_hat.apply(params, jnp.array((x, t)))[0],
                x[0], x[1]  # this is for x and t. No better way exists to do this
            )
            return error
        return jnp.mean(jax.vmap(loss_unit)(xs))
    return jax.jit(loss)

def make_max_loss(expression):
    u_hat, _ = expression.u()
    # hyper param, num of samples per loss
    xs = expression.matrix(10)
    def loss(params):
        def loss_unit(x):
            error = expression.loss(
                lambda x, t: u_hat.apply(params, jnp.array((x, t)))[0],
                x[0], x[1]  # this is for x and t. No better way exists to do this
            )
            return error
        return jnp.max(jax.vmap(loss_unit)(xs))
    return jax.jit(loss)

# 
n_10_mean_loss = list()
n_10_mean_time = list()
n_10_max_loss = list()
n_10_max_time = list()

for i in tqdm(range(100)):
    print("n = 10")
    start = time.time()
    heat_loss = make_mean_loss(heat)
    heat_loss, grads = jax.value_and_grad(heat_loss)(params)
    end = time.time()
    
    n_10_mean_loss.append(heat_loss)
    n_10_mean_time.append(end-start)
    
    start = time.time()
    heat_loss = make_max_loss(heat)
    heat_loss, grads = jax.value_and_grad(heat_loss)(params)
    end = time.time()
    
    n_10_max_loss.append(heat_loss)
    n_10_max_time.append(end-start)
    
#.  n = 30

def make_mean_loss(expression):
    u_hat, _ = expression.u()
    # hyper param, num of samples per loss
    xs = expression.matrix(30)
    def loss(params):
        def loss_unit(x):
            error = expression.loss(
                lambda x, t: u_hat.apply(params, jnp.array((x, t)))[0],
                x[0], x[1]  # this is for x and t. No better way exists to do this
            )
            return error
        return jnp.mean(jax.vmap(loss_unit)(xs))
    return jax.jit(loss)

def make_max_loss(expression):
    u_hat, _ = expression.u()
    # hyper param, num of samples per loss
    xs = expression.matrix(30)
    def loss(params):
        def loss_unit(x):
            error = expression.loss(
                lambda x, t: u_hat.apply(params, jnp.array((x, t)))[0],
                x[0], x[1]  # this is for x and t. No better way exists to do this
            )
            return error
        return jnp.max(jax.vmap(loss_unit)(xs))
    return jax.jit(loss)

# 
n_30_mean_loss = list()
n_30_mean_time = list()
n_30_max_loss = list()
n_30_max_time = list()

for i in tqdm(range(100)):
    print("n = 30")
    start = time.time()
    heat_loss = make_mean_loss(heat)
    heat_loss, grads = jax.value_and_grad(heat_loss)(params)
    end = time.time()
    
    n_30_mean_loss.append(heat_loss)
    n_30_mean_time.append(end-start)
    
    start = time.time()
    heat_loss = make_max_loss(heat)
    heat_loss, grads = jax.value_and_grad(heat_loss)(params)
    end = time.time()
    
    n_30_max_loss.append(heat_loss)
    n_30_max_time.append(end-start)

# displaying

n_30_mean_loss_arr = jnp.array(n_30_mean_loss)
n_30_mean_time_arr = jnp.array(n_30_mean_time)
n_30_max_loss_arr = jnp.array(n_30_max_loss)
n_30_max_time_arr = jnp.array(n_30_max_time)

n_10_mean_loss_arr = jnp.array(n_10_mean_loss)
n_10_mean_time_arr = jnp.array(n_10_mean_time)
n_10_max_loss_arr = jnp.array(n_10_max_loss)
n_10_max_time_arr = jnp.array(n_10_max_time)

# Compute mean and standard deviation for n = 30
n_30_mean_loss_mean = jnp.mean(n_30_mean_loss_arr)
n_30_mean_loss_std = jnp.std(n_30_mean_loss_arr)
n_30_mean_time_mean = jnp.mean(n_30_mean_time_arr)
n_30_mean_time_std = jnp.std(n_30_mean_time_arr)

n_30_max_loss_mean = jnp.mean(n_30_max_loss_arr)
n_30_max_loss_std = jnp.std(n_30_max_loss_arr)
n_30_max_time_mean = jnp.mean(n_30_max_time_arr)
n_30_max_time_std = jnp.std(n_30_max_time_arr)

# Compute mean and standard deviation for n = 10
n_10_mean_loss_mean = jnp.mean(n_10_mean_loss_arr)
n_10_mean_loss_std = jnp.std(n_10_mean_loss_arr)
n_10_mean_time_mean = jnp.mean(n_10_mean_time_arr)
n_10_mean_time_std = jnp.std(n_10_mean_time_arr)

n_10_max_loss_mean = jnp.mean(n_10_max_loss_arr)
n_10_max_loss_std = jnp.std(n_10_max_loss_arr)
n_10_max_time_mean = jnp.mean(n_10_max_time_arr)
n_10_max_time_std = jnp.std(n_10_max_time_arr)

# Print results in a summary table format, including the actual statistics
print(f"{'Sample Size':<12} {'Method':<6} {'Metric':<5} {'Mean':<10} {'Standard Deviation':<10}")
print(f"{'n = 30':<12} {'Mean':<6} {'Loss':<5} {n_30_mean_loss_mean:<10.4f} {n_30_mean_loss_std:<10.4f}")
print(f"{'n = 30':<12} {'Mean':<6} {'Time':<5} {n_30_mean_time_mean:<10.4f} {n_30_mean_time_std:<10.4f}")
print(f"{'n = 10':<12} {'Mean':<6} {'Loss':<5} {n_10_mean_loss_mean:<10.4f} {n_10_mean_loss_std:<10.4f}")
print(f"{'n = 10':<12} {'Mean':<6} {'Time':<5} {n_10_mean_time_mean:<10.4f} {n_10_mean_time_std:<10.4f}")
print()
print(f"{'n = 30':<12} {'Max':<6} {'Loss':<5} {n_30_max_loss_mean:<10.4f} {n_30_max_loss_std:<10.4f}")
print(f"{'n = 30':<12} {'Max':<6} {'Time':<5} {n_30_max_time_mean:<10.4f} {n_30_max_time_std:<10.4f}")
print(f"{'n = 10':<12} {'Max':<6} {'Loss':<5} {n_10_max_loss_mean:<10.4f} {n_10_max_loss_std:<10.4f}")
print(f"{'n = 10':<12} {'Max':<6} {'Time':<5} {n_10_max_time_mean:<10.4f} {n_10_max_time_std:<10.4f}")

  0%|                                                                                                                 | 0/100 [00:00<?, ?it/s]

n = 10


  1%|█                                                                                                        | 1/100 [00:10<17:15, 10.46s/it]

n = 10


  2%|██                                                                                                       | 2/100 [00:20<16:14,  9.95s/it]

...

n = 30


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [10:35<00:06,  6.29s/it]

n = 30


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:43<00:00,  6.44s/it]


Sample Size  Method Metric Mean       Standard Deviation
n = 30       Mean   Loss  1.3118     0.0255    
n = 30       Mean   Time  3.2682     0.6160    
n = 10       Mean   Loss  1.3107     0.0378    
n = 10       Mean   Time  4.6720     0.2817    

n = 30       Max    Loss  1.6824     0.0572    
n = 30       Max    Time  3.1670     0.2651    
n = 10       Max    Loss  1.5994     0.1142    
n = 10       Max    Time  4.6328     0.1850    


In [32]:
# loss metrics